<a href="https://colab.research.google.com/github/FabianVCha/4_en_liena/blob/main/segmentation_algoritm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/facebookresearch/segment-anything-2.git
%cd segment-anything-2/
!pip install -e .

Cloning into 'segment-anything-2'...
remote: Enumerating objects: 948, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 948 (delta 4), reused 6 (delta 2), pack-reused 936 (from 1)
Receiving objects: 100% (948/948), 128.92 MiB | 17.13 MiB/s, done.
Resolving deltas: 100% (321/321), done.
/content/segment-anything-2
Obtaining file:///content/segment-anything-2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!/content/segment-anything-2/checkpoints/download_ckpts.sh

--2024-10-03 22:44:36--  https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_tiny.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.102, 108.157.254.121, 108.157.254.124, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.254.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 156008466 (149M) [application/vnd.snesdev-page-table]
Saving to: ‘sam2.1_hiera_tiny.pt’

sam2.1_hiera_tiny.p 100%[===================>] 148.78M   293MB/s    in 0.5s    

2024-10-03 22:44:36 (293 MB/s) - ‘sam2.1_hiera_tiny.pt’ saved [156008466/156008466]

--2024-10-03 22:44:36--  https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_small.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.102, 108.157.254.121, 108.157.254.124, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.254.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18

In [3]:
%cd segment-anything-2/
!python setup.py build_ext --inplace

[Errno 2] No such file or directory: 'segment-anything-2/'
/content/segment-anything-2
running build_ext
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:495: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:414: UserWarning: The detected CUDA version (12.2) has a minor version mismatch with the version that was used to compile PyTorch (12.1). Most likely this shouldn't be a problem.
  warnings.warn(CUDA_MISMATCH_WARN.format(cuda_str_version, torch.version.cuda))
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:424: UserWarning: There are no x86_64-linux-gnu-g++ version bounds defined for CUDA version 12.2
  warnings.warn(f'There are no {compiler_name} version bounds defined for CUDA version {cuda_str_version}')
building 'sam2._

In [2]:
%cd /content/segment-anything-2

/content/segment-anything-2


In [4]:
import os
import torch
import base64
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [5]:
!pip install supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 11.7 MB/s eta 0:00:00


In [7]:
!mkdir /content/checkpoints
!wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_tiny.pt -P /content/checkpoints
!wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_small.pt -P /content/checkpoints
!wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_base_plus.pt -P /content/checkpoints
!wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt -P /content/checkpoints

In [8]:
from sam2.build_sam import build_sam2_video_predictor

CHECKPOINT = "/content/checkpoints/sam2_hiera_base_plus.pt"
CONFIG = "sam2_hiera_b+.yaml"

sam2_model = build_sam2_video_predictor(CONFIG, CHECKPOINT)

In [ ]:
!mkdir /content/frame

import supervision as sv

frames_generator = sv.get_video_frames_generator('/content/darkos2.mp4')
sink = sv.ImageSink(
    target_dir_path='/content/frame',
    image_name_pattern="{:05d}.jpeg")
with sink:
    for frame in frames_generator:
        sink.save_image(frame)

In [ ]:
inference_state = sam2_model.init_state('/content/frame')

frame loading (JPEG): 100%|██████████| 259/259 [00:12<00:00, 20.08it/s]


In [ ]:
sam2_model.reset_state(inference_state)

In [ ]:
def encode_image(filepath):
  with open(filepath, 'rb') as f:
    image_bytes = f.read()
  encoded = str(base64.b64encode(image_bytes), 'utf-8')
  return "data:image/jpg;base64,"+encoded

In [ ]:
!pip install jupyter_bbox_widget

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.4/727.4 kB 43.6 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [ ]:
IS_COLAB = True

if IS_COLAB:
  from google.colab import output
  output.enable_custom_widget_manager()

from jupyter_bbox_widget import BBoxWidget

widget = BBoxWidget()
widget.image = encode_image('/content/frame/00000.jpeg')
widget

BBoxWidget(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#b…

In [ ]:
widget.bboxes

[{'x': 432, 'y': 565, 'width': 0, 'height': 1, 'label': ''}]

In [ ]:
import numpy as np

points = np.array([[432, 565]], dtype=np.float32)
labels = np.array([1])
frame_idx = 0
tracker_id = 1

_, object_ids, mask_logits = sam2_model.add_new_points(
    inference_state=inference_state,
    frame_idx=frame_idx,
    obj_id=tracker_id,
    points=points,
    labels=labels,
)

In [ ]:
!pip install imageio

In [ ]:
import cv2
import numpy as np
import supervision as sv

# Definir los colores de la máscara
colors = ['#FF1493', '#00BFFF', '#FF6347', '#FFD700']
color_palette = [tuple(int(color[i:i+2], 16) for i in (1, 3, 5)) for color in colors]  # Convertir hex a BGR

# Información del video y los frames
video_info = sv.VideoInfo.from_video_path('/content/darkos2.mp4')
frames_paths = sorted(sv.list_files_with_extensions(
    directory='/content/frame',
    extensions=["jpeg"])
)
# Función para quitar el fondo del frame y establecerlo a gris
def remove_background(frame, mask):
    # Asegurarse de que la máscara esté en el formato correcto (0 y 1)
    mask = (mask > 0).astype(np.uint8)

    # Mantener el objeto
    object_segment = cv2.bitwise_and(frame, frame, mask=mask)  # Solo el objeto segmentado

    # Crear una imagen de fondo gris (#808080)
    gray_background = np.full(frame.shape, fill_value=(0, 0, 0), dtype=np.uint8)  # Fondo gris
    # Alternativa usando el valor hexadecimal
    # gray_background = np.full(frame.shape, fill_value=(0x80, 0x80, 0x80), dtype=np.uint8)

    # Combinar el objeto con el fondo gris
    result = cv2.addWeighted(gray_background, 1, object_segment, 1, 0)  # Mezclar el fondo gris y el objeto
    return result

# Abrir el sink para guardar el video procesado
with sv.VideoSink('/content/video_segmented_3.mp4', video_info=video_info) as sink:
    for frame_idx, object_ids, mask_logits in sam2_model.propagate_in_video(inference_state):
        # Cargar el frame actual
        frame = cv2.imread(frames_paths[frame_idx])

        # Mover las máscaras a la GPU y procesarlas en la GPU
        mask_logits = mask_logits.to('cuda')  # Asegúrate de que los tensores estén en la GPU
        masks = (mask_logits > 0.0).cpu().numpy()  # Volver a la CPU solo cuando sea necesario

        # Invertir las máscaras si es necesario
        N, X, H, W = masks.shape
        masks = masks.reshape(N * X, H, W)

        # Usar la máscara para quitar el fondo del frame y establecerlo a gris
        frame_no_bg = remove_background(frame, masks[0])  # Usamos la primera máscara

        # Asegurarse de que la imagen que se pasa a annotate tenga solo tres canales
        frame_rgb = frame_no_bg[:, :, :3]  # Obtener solo los canales RGB
        frame_rgb = frame_rgb.astype(np.uint8)  # Asegurarse de que sea de tipo uint8

        # Anotar el frame
        detections = sv.Detections(
            xyxy=sv.mask_to_xyxy(masks=masks),
            mask=masks,
            tracker_id=np.array(object_ids)
        )

        # Crear la máscara de anotación
        colored_mask = np.zeros_like(frame_rgb, dtype=np.uint8)

        for idx in range(len(detections.tracker_id)):
            # Obtener el color correspondiente del color de la máscara
            tracker_id = detections.tracker_id[idx]
            color = color_palette[tracker_id % len(color_palette)]  # Seleccionar color basado en el ID
            color_bgr = (color[2], color[1], color[0])  # Convertir a BGR

            # Aplicar el color solo si no es azul
            if color_bgr == (255, 191, 0):  # Suponiendo que el azul es (255, 191, 0) en BGR
                continue  # No hacer nada, se mantendrá transparente
            else:
                mask_idx = detections.mask[idx] > 0  # Obtener la máscara del objeto
                colored_mask[mask_idx] = color_bgr  # Aplicar el color, ignorando el canal alfa
        # Aplicar la máscara al frame
        frame_rgb = cv2.addWeighted(frame_rgb, 1, colored_mask, 0.5, 0)
        # Guardar el frame con el fondo gris
        sink.write_frame(frame_rgb)


propagate in video: 100%|██████████| 259/259 [02:16<00:00,  1.90it/s]


In [12]:
import cv2
import os
from google.colab import files
from IPython.display import HTML

# Cargar el modelo de detección de rostros preentrenado de OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Cargar el video (asegúrate de subir un archivo de video a Colab)
input_video_path = '/content/ruta_al_video.mp4'  # Reemplaza con la ruta al video subido
output_video_path = '/content/video_censurado.mp4'

# Abrir el video original
video = cv2.VideoCapture(input_video_path)

# Obtener las propiedades del video
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))

# Definir el codec y crear el objeto para guardar el video de salida
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec de video
output = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar rostros en el frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extraer la región del rostro detectado
        face_region = frame[y:y+h, x:x+w]

        # Aplicar un desenfoque en la región del rostro
        face_region = cv2.GaussianBlur(face_region, (99, 99), 30)

        # Reemplazar la región censurada en el frame original
        frame[y:y+h, x:x+w] = face_region

    # Guardar el frame censurado en el video de salida
    output.write(frame)

# Liberar los objetos
video.release()
output.release()

In [13]:
!pip install facenet-pytorch opencv-python-headless

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 101.1 MB/s eta 0:0

In [14]:
import torch
import cv2
import imageio
import numpy as np
from facenet_pytorch import MTCNN
from google.colab import files

# Inicializar el detector MTCNN
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(keep_all=True, device=device)

# Cargar el video (asegúrate de subir un archivo de video a Colab)
input_video_path = '/content/video_segmented_4.mp4'  # Reemplaza con la ruta de tu video
output_gif_path = '/content/video_censurado_mtcnn.mp4'

# Abrir el video original
video = cv2.VideoCapture(input_video_path)

# Obtener las propiedades del video (puedes reducir la resolución para mayor velocidad)
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH)) // 2  # Reducimos la resolución
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)) // 2
fps = int(video.get(cv2.CAP_PROP_FPS))  # Mantengo los FPS originales

# Crear una lista para almacenar los frames procesados para el GIF
frames = []

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    # Redimensionar el frame para optimizar velocidad
    frame = cv2.resize(frame, (frame_width, frame_height))

    # Convertir el frame a RGB (MTCNN requiere RGB)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detección de rostros usando MTCNN
    boxes, _ = mtcnn.detect(frame_rgb)

    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)

            # Calcular el centro y reducir el radio a la mitad
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            radius = int(max((x2 - x1) // 2, (y2 - y1) // 2) *0.75)  # Dividir el radio por 2 para reducirlo a la mitad

            # Crear una máscara circular con el radio reducido
            mask = np.zeros_like(frame, dtype=np.uint8)
            cv2.circle(mask, (center_x, center_y), radius, (255, 255, 255), -1)

            # Crear una copia del frame para desenfocar solo la parte dentro del círculo
            blurred_frame = cv2.GaussianBlur(frame, (99, 99), 30)

            # Usar la máscara circular para combinar el frame original con el desenfocado
            frame = np.where(mask == 255, blurred_frame, frame)

    # Guardar el frame procesado en la lista de frames
    frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))  # Guardar como RGB para el GIF

# Liberar el video
video.release()

# Crear y guardar el GIF a partir de los frames procesados
imageio.mimsave(output_gif_path, frames, fps=fps)

# Descargar el GIF generado
files.download(output_gif_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>